### Построение линейной композиции моделей с использованием сиплекс-метода

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans
import scipy.stats as sps
from copy import deepcopy

In [2]:
import receive_data

import sys
import os
CUR_DIR = os.path.dirname(os.path.abspath("./")) + "/"

sys.path.append(CUR_DIR + 'genetic_algorithm')
import mutation
import crossover
import create_population
from mutation import mutate_rand_tree

In [3]:
from math import sin

def f_sin(x, y):
    return sin(x + y)

In [4]:
sin_sample = np.array([sps.uniform(-10, 20).rvs(1000), sps.uniform(-10, 20).rvs(1000)]).T
sin_sample_real = np.vectorize(f_sin)(sin_sample[:, 0], sin_sample[:, 1])

In [5]:
clusterizer = KMeans()

clusterizer.fit(sin_sample)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=8, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [6]:
cluster_nums = clusterizer.predict(sin_sample)
samples = []
reals = []

for i in range(clusterizer.n_clusters):
    samples.append(sin_sample[cluster_nums == i])
    reals.append(sin_sample_real[cluster_nums == i])

In [10]:
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

def genetic_algo_mse(sample, real, iter_cnt=20, values_history=None):
    np.random.seed(42)

    size = 20

    population = create_population.create_population(size, 5)

  #  history = []

    for iteration in range(iter_cnt):
        new_population = []

        sz = len(population)

        new_population.append(population[0])

        for i in range(sz // 30):
            new_population.append(mutate_rand_tree(population[0]))
            new_population.append(mutate_rand_tree(population[1]))
            new_population.append(mutate_rand_tree(population[2]))
        for i in range(sz // 10):
            new_population.append(create_population.create_random_model(4))
        for i in range(sz):
            t1 = population[np.random.randint(0, sz / 2)]
            t2 = population[np.random.randint(0, sz / 2)]
            new_population.append(mutate_rand_tree(crossover.crossfit(t1, t2)))

        population = np.array(list(population) + list(new_population))
        values = []
        for func in population:
            #print(np.vectorize(func.calc)(sample[:, 0], sample[:, 1]))
            values.append(mean_squared_error(np.vectorize(func.calc)(sample[:, 0], sample[:, 1]), real))
        values = np.array(values)

        indexes = np.arange(len(population))
        indexes = sorted(indexes, key=lambda i: values[i])

        population = (population[indexes])[:size]
        values = (values[indexes])[:size]

       # vis(population, values)
       # history.append((population, values))

    print(values[0])
    return population[0]
        

Для начала построим суперпозицию сразу на всей обучающей выборке.

In [11]:
general_model = genetic_algo_mse(sin_sample, sin_sample_real)

/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:76: RuntimeWarning: invalid value encountered in double_scalars
  vars = np.array([fr.low * sc.low, fr.low * sc.high, fr.high * sc.low, fr.high * sc.high])
/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:84: RuntimeWarning: invalid value encountered in double_scalars
  vars = np.array([fr.low * sc.low, fr.low * sc.high, fr.high * sc.low, fr.high * sc.high])
/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:52: RuntimeWarning: overflow encountered in exp
  return self.get_correct_value(self.func(*[node.calc(x, y) for node in self.nodes]))
/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:52: RuntimeWarning: invalid value encountered in sqrt
  return self.get_correct_value(self.func(*[node.calc(x, y) 

0.4983214637139154


In [96]:
import create_population 

genetic_result_list = []
model_list = []

for (sample, real) in zip(samples, reals):
    population = create_population.create_population(30, 5)
    model_list.append(genetic_algo_mse(sample, real))

/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:84: RuntimeWarning: invalid value encountered in double_scalars
  vars = np.array([fr.low * sc.low, fr.low * sc.high, fr.high * sc.low, fr.high * sc.high])
/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:76: RuntimeWarning: invalid value encountered in double_scalars
  vars = np.array([fr.low * sc.low, fr.low * sc.high, fr.high * sc.low, fr.high * sc.high])
/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:52: RuntimeWarning: overflow encountered in exp
  return self.get_correct_value(self.func(*[node.calc(x, y) for node in self.nodes]))
/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:52: RuntimeWarning: invalid value encountered in sqrt
  return self.get_correct_value(self.func(*[node.calc(x, y) 

0.438921188057297


/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:52: RuntimeWarning: invalid value encountered in true_divide
  return self.get_correct_value(self.func(*[node.calc(x, y) for node in self.nodes]))


0.4798112168297868
0.48171446410103197
0.44952674615030824
0.4678029068751508
0.517228387777265
0.4907214956577084
0.45274689026643766


In [97]:
from scipy.optimize import minimize, fmin

In [121]:
def superposition_model(x, y, weights):
    return sum([weights[i] * model_list[i].calc(x, y) for i in range(len(model_list))])

def superposition_score(weights):
    linear_superposition = lambda x, y: superposition_model(x, y, weights)
    return mean_squared_error(np.vectorize(linear_superposition)(sin_sample[:, 0], sin_sample[:, 1]), sin_sample_real)

def kek_fun(x):
    return (x[0] - 10.0)**2 + np.abs(x[1])

In [131]:
x0 = np.zeros(len(model_list))
print(x0)

minimum = fmin(
    superposition_score, 
    x0,
    maxiter=100,
    maxfun=1000
)

[0. 0. 0. 0. 0. 0. 0. 0.]


/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:52: RuntimeWarning: invalid value encountered in sqrt
  return self.get_correct_value(self.func(*[node.calc(x, y) for node in self.nodes]))
/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:171: RuntimeWarning: invalid value encountered in log10
  LOG = Primitive(lambda x: np.log10(1+x), 1, DOMAINS.POSITIVE, DOMAINS.REAL, 'log')


Optimization terminated successfully.
         Current function value: 0.499013
         Iterations: 53
         Function evaluations: 99


In [133]:
print(minimum, superposition_score(minimum))

/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:52: RuntimeWarning: invalid value encountered in sqrt
  return self.get_correct_value(self.func(*[node.calc(x, y) for node in self.nodes]))
/home/mike/Документы/MIPT_sem6/my_science/2019-Project-27/code/base_experiment/genetic_algorithm/primitive.py:171: RuntimeWarning: invalid value encountered in log10
  LOG = Primitive(lambda x: np.log10(1+x), 1, DOMAINS.POSITIVE, DOMAINS.REAL, 'log')


[-1.08100717e-05 -4.29574673e-07  1.02891336e-04 -5.22080906e-06
 -9.64660614e-07  8.91853698e-06  1.34967350e-04 -3.67513813e-07] 0.4990128120629202
